In [ ]:
import sys
import pdb
import pandas as pd
from sklearn import tree
from sklearn import ensemble
import numpy as np
import numpy.random as rn
import matplotlib.pyplot as plt
import os
import urllib.request
import io
import zipfile
import datetime
## Reading Data
from functions import *

In [ ]:
from statistics import mean
def score(REAL_LABELS, PREDICTED_LABELS,var_rf,var_rnn,var_merge):
    predicter = list(range(len(var_rf)))
    a1 = pd.DataFrame(index=range(len(var_rf)), columns=range(2))
    a1.columns = ['index', 'value']
    a2 = pd.DataFrame(index=range(len(var_rf)), columns=range(2))
    a2.columns = ['index', 'value']
    for i in range(len(predicter)):
        if i in PREDICTED_LABELS:
            a1.iloc[i, 1] = 1
        else:
            a1.iloc[i, 1] = 0
    for i in range(len(predicter)):
        if i in REAL_LABELS:
            a2.iloc[i, 1] = 1
        else:
            a2.iloc[i, 1] = 0
    y_real = a2.value
    y_real = y_real.astype(int)
    y_predi = a1.value
    y_predi = y_predi.astype(int)
    cm = confusion_matrix(y_true=y_real, y_pred=y_predi)
    tp = cm[0][0]
    fp = cm[0][1]
    fn = cm[1][0]
    tn = cm[1][1]
    precision1 = tp / (tp + fp)
    recall1 = tp / (tp + fn)
    Accuracy1 = (tp + tn) / len(var_rf)
    F11 = 2 / ((1 / precision1) + (1 / recall1))
    return precision1, recall1, Accuracy1, F11

In [ ]:
def voting(anomalies_rf, anomalies_rnn, anomalies_merged):
  #  anomalies_rf['weight'] = var_rf
  #  anomalies_rnn['weight'] = var_rnn
  #  anomalies_merged['weight'] = var_merge
   # anomalies_merged = anomalies_merged.sort_values(by='index')
    #anomalies_merged['weight'] = anomalies_merged['weight'].cumsum()
    #anomalies_merged['weight'] = anomalies_merged['weight'] / anomalies_merged['weight'].max()
    #anomalies_merged['weight'] = anomalies_merged['weight'].round(3)

In [ ]:
    anomalies_rnn1=anomalies_rnn.values.tolist()
    anomalies_rf1=anomalies_rf.values.tolist()
    anomalies=set(anomalies_rf1+anomalies_rnn1)
    anomalies=list(anomalies)
    anomalies=set(anomalies + anomalies_merged)
    #anomalies=pd.DataFrame(anomalies)
    anomalies=list(anomalies)

In [ ]:
    #anomalies.columns=['index']
    #anomalies_rf=pd.DataFrame(anomalies_rf1)
    #anomalies_rf.columns=['index']
    #anomalies_rnn=pd.DataFrame(anomalies_rnn1)
    #anomalies_rnn.columns=['index']
    #anomalies_merged=pd.DataFrame(anomalies_merged)
    #anomalies_merged.columns=['index']

In [ ]:
    append_anomalies=[]
    for i in range(len(anomalies)):
        if anomalies[i] in anomalies_rf and  anomalies[i] in anomalies_rnn and anomalies[i] in anomalies_merged:
            append_anomalies.append(anomalies[i])
        elif  anomalies[i] in anomalies_rnn and  anomalies[i] in anomalies_merged:
            append_anomalies.append(anomalies[i])
        elif anomalies[i] in anomalies_rf and  anomalies[i] in anomalies_merged:
                append_anomalies.append(anomalies[i])
        elif anomalies[i] in anomalies_rf and  anomalies[i] in anomalies_rnn:
            append_anomalies.append(anomalies[i])
    return append_anomalies

In [ ]:
DATA_URL = 'https://s3-us-west-2.amazonaws.com/telemanom/data.zip'

In [ ]:
Label_URL = 'https://github.com/khundman/telemanom/raw/master/labeled_anomalies.csv'

In [ ]:
df_label = pd.read_csv(Label_URL)

In [ ]:
MSL=df_label[df_label.spacecraft=='MSL']['chan_id']
SMAP=df_label[df_label.spacecraft=='SMAP']['chan_id']
train_signals=SMAP
#print(SMAP)

In [ ]:
if not os.path.exists('data'):
    response = urllib.request.urlopen(DATA_URL)
    bytes_io = io.BytesIO(response.read())
    with zipfile.ZipFile(bytes_io) as zf:
        zf.extractall()

In [ ]:
train_signals = os.listdir('data/train')
test_signals = os.listdir('data/test')

fix random seed for reproducibility

In [ ]:
np.random.seed(0)

In [ ]:
os.makedirs('csv', exist_ok=True)
im=0
#pdb.set_trace()
# %%
training_truth_df=pd.DataFrame()
df_label=df_label[df_label['chan_id'].isin(SMAP)]

In [ ]:
df_label=df_label.reset_index(drop=True)
selected_columns = ['value']
precision_rf=[]
recall_rf=[]
Accuracy_rf=[]
F1_rf=[]
precision_rnn=[]
recall_rnn=[]
Accuracy_rnn=[]
F1_rnn=[]
precision_merge=[]
recall_merge=[]
Accuracy_merge=[]
F1_merge=[]
precision_voting=[]
recall_voting=[]
Accuracy_voting=[]
F1_voting=[]
#SMAP = ['P-1']

In [ ]:
SMAP=[  'P-1', 'S-1', 'E-1', 'E-2','E-5', 'E-6', 'E-7',
    'E-8', 'E-9', 'E-10', 'E-11', 'E-12', 'E-13',   'P-3',
     'A-2', 'A-4', 'G-2',
    'D-7', 'F-1', 'P-4', 'G-3', 'T-1', 'T-2', 'D-8', 'D-9',
    'G-4', 'T-3',  'D-12', 'B-1',  'G-7', 'P-7',
     'A-5', 'A-7', 'D-13',  'A-9']

In [ ]:
MSL=[
    'M-1', 'S-2',  'T-4', 'T-5', 'F-7',  'M-4',
    'M-5',  'C-1', 'C-2', 'T-12', 'T-13', 'F-4', 'F-5', 'D-14',
    'T-9',  'T-8',  'D-15', 'M-7', 'F-8']

In [ ]:
for name in SMAP:
    label_row = df_label[df_label.chan_id == name]
   # pdb.set_trace()
    labels = label_row.anomaly_sequences[im]
    labels = eval(labels)
    true_indices_grouped = [list(range(e[0], e[1] + 1)) for e in labels]
    true_indices_flat =set([i for group in true_indices_grouped for i in group])
    appended_data = []

In [ ]:
    labels = true_indices_flat

In [ ]:
    index = list(labels)

    #timestamp = index * 86400 + 1022819200
    anomalies = pd.DataFrame({'value': 1, 'index': index})
    appended_data.append(anomalies)
    label_data = anomalies

In [ ]:
 #label_data ['date'] = pd.to_datetime(label_data ['timestamp'], unit='s')
    # label_data ['month'] = label_data ['date'].dt.month.astype(int)
    #label_data['name'] = name
    # label_data ['day_of_week'] = label_data ['date'].dt.dayofweek.astype(int)
    # label_data ['hour_of_day'] = label_data ['date'].dt.hour.astype(int)
    #label_data = label_data[selected_columns]
    # label_data .to_csv('csv/' + name + '.csv', index=False)
    signal = name
    train_np = np.load('data/train/' + signal + '.npy')
    test_np = np.load('data/test/' + signal + '.npy')
    data = build_df(np.concatenate([train_np, test_np]))
    #data['date'] = pd.to_datetime(data['timestamp'], unit='s')
    #data['month'] = data['date'].dt.month.astype(int)
    data['name'] = name
    data['index'] = data['index'].astype(int)
    #data['day_of_week'] = data['date'].dt.dayofweek.astype(int)
    #data['hour_of_day'] = data['date'].dt.hour.astype(int)
    data = data[selected_columns]
    data.to_csv('csv/' + name + '.csv', index=False)
    anomalies_rf, anomalies_rnn, anomalies_merged,var_merge,var_rf,var_rnn=evaluate_ga('../csv/'+name+'.csv')
    anomaly_voting=voting(anomalies_rf, anomalies_rnn, anomalies_merged)
    precision1_rf, recall1_rf, Accuracy1_rf, F11v=score(label_data,anomalies_rf,anomaly_voting,anomalies_rnn,anomalies_merged)
    precision1_rnn, recall1_rnn, Accuracy1_rnn, F11_rnn = score(label_data, anomalies_rnn, anomalies_rf, anomaly_voting,
                                                anomalies_merged)
    precision1_merge, recall1_merge, Accuracy1_merge, F11_merge = score(label_data, anomalies_merged, anomalies_rf, anomalies_rnn,
                                                anomaly_voting)
    precision1_voting, recall1_voting, Accuracy1_voting, F11_voting = score(label_data, anomaly_voting, anomalies_rf, anomalies_rnn,
                                                      anomalies_merged)
    precision_rf.append(precision1_rf)
    recall_rf.append(recall1_rf)
    Accuracy_rf.append(Accuracy1_rf)
    F1_rf.append(F11v)
    precision_rnn.append(precision1_rnn)
    recall_rnn.append(recall1_rnn)
    Accuracy_rnn.append(Accuracy1_rnn)
    F1_rnn.append(F11_rnn)
    precision_merge.append(precision1_merge)
    recall_merge.append(recall1_merge)
    Accuracy_merge.append(Accuracy1_merge)
    F1_merge.append(F11_merge)
    precision_voting.append(precision1_voting)
    recall_voting.append(recall1_voting)
    Accuracy_voting.append(Accuracy1_voting)
    F1_voting.append(F11_voting)
    im+=1
    print(name)

In [ ]:
recall_final_rnn=mean(recall_rnn)
precision_final_rnn=mean(precision_rnn)
F1_final_rnn=mean(F1_rnn)
Accuracy_final_rnn=mean(Accuracy_rnn)

In [ ]:
recall_final_rf=mean(recall_rf)
precision_final_rf=mean(precision_rf)
F1_final_rf=mean(F1_rf)
Accuracy_final_rf=mean(Accuracy_rnn)

In [ ]:
recall_final_merge=mean(recall_merge)
precision_final_merge=mean(precision_merge)
F1_final_merge=mean(F1_merge)
Accuracy_final_merge=mean(Accuracy_merge)

In [ ]:
recall_final_voting=mean(recall_voting)
precision_final_voting=mean(precision_voting)
F1_final_voting=mean(F1_voting)
Accuracy_final_voting=mean(Accuracy_voting)

In [ ]:
print('recall_final_rnn',recall_final_rnn)
print('precision_final_rnn',precision_final_rnn)
print('F1_final_rnn',F1_final_rnn)
print('Accuracy_final_rnn',Accuracy_final_rnn)
print('recall_final_rf',recall_final_rf)
print('precision_final_rf',precision_final_rf)
print('F1_final_rf',F1_final_rf)
print('Accuracy_final_rf',Accuracy_final_rf)
print('recall_final_merge',recall_final_merge)
print('precision_final_merge',precision_final_merge)
print('F1_final_merge',F1_final_merge)
print('Accuracy_final_merge',Accuracy_final_merge)
print('recall_final_voting',recall_final_voting)
print('precision_final_voting',precision_final_voting)
print('F1_final_voting',F1_final_voting)
print('Accuracy_final_voting',Accuracy_final_voting)